## 1.1 - Uma estrutura de diretórios e nomenclatura de arquivos que permita armazenar as consultas de modo coerente

Utilizando as boas práticas, separaria cada fase dos dados por camadas, sendo as 3 principais: bronze (dados em seu estado puro, uma copia e cola da origem), silver (dados com alguns tratamentos simples - correções de nomes, validações de nomes, validações de dados nulls) e gold (dados em seu estado final e pronto para carregamento).

Na camada bronze, teríamos a partição por país, sendo o diretório inicial o país de origem, dentro do diretório inicial teríamos pastas que seriam os países de destino e dentro dos países de destino teríamos uma partição por ano. Em relação a nomenclatura, utilizariamos as 3 siglas iniciais dos países. Ou seja, seria algo semelhante a "/bronze/bra/arg/2019.txt"

Na camada silver, teríamos a partição por país, sendo o diretório inicial o país de origem, dentro do diretório inicial teríamos pastas que seriam os países de destino e dentro dos países de destino teríamos uma partição por ano. Em relação a nomenclatura, utilizariamos as 3 siglas iniciais dos países, sendo que neste nível seria ideal salvar os dados como um arquivo parquet. Ou seja, seria algo semelhante a "/silver/bra/arg/2019.parquet"

Na camada gold, teríamos a partição por país, sendo o diretório inicial o país de origem, dentro do diretório inicial teríamos pastas que seriam os países de destino e dentro dos países de destino teríamos uma partição por ano. Em relação a nomenclatura, utilizariamos as 3 siglas iniciais dos países, ou seja, seria algo semelhante a "/gold/bra/arg/2019.parquet"

## 1.2 - Liste as etapas necessárias para integração de 3 anos (2019~2021) de comércio exterior

1 - Realizar o request em todos os países de origem, destino e anos

In [ ]:
import requests as req

resultado = req.get('https://link.api/dados')

lista_paises = []

for pais in resultado.json()['pais_origem']:
    lista_paises.append(pais)

anos_considerados = ['2019', '2020', '2021']
dados = {}
dados_filtrados = []

for pais_origem in lista_paises:
    for pais_destino in lista_paises:
        if pais_origem == pais_destino:
            continue
        for ano in resultado.json()['ano']:
            for ano['ano'] in anos_considerados:
                dados = {
                    'pais_origem': pais_origem,
                    'pais_destino': pais_destino,
                    'ano': ano['ano']
                }
                dados_filtrados.append(dados)


2- Salva os dados na camada bronze

3- Realiza ajustes necessários nos dados: correção de valores nulos, correção de valores em branco, tratamento de tipos de dados

4- Salva os dados na camada silver

5- Realiza cálculos necessários pertinentes ao projeto

6- Salva os dados na camada gold

## 1.3 - Qual/Quais arquivo(s) da sua base de dados respondem a questão: 'Qual a quantidade de soja o Mundo importou do Brasil em 2020?'

O arquivo estaria na pasta "gold/mun/bra/2020.parquet"

## 1.4 – Sua base de dados alimenta um painel que possui informações de comércio exterior entre países – e também mundo. As consultas de comércio exterior podem ser feitas com os parâmetros M (‘mês’) ou A(‘ano’). A consulta M representa um report dos meses disponíveis no ano até então. A consulta A representa o report final/completo das imp/exp em determinado ano. Considere que cada país escolhe uma data “aleatória” do primeiro trimestre do novo ano para realizar o report final/completo das suas exportações e importações anuais do ano anterior.
## Liste as etapas necessárias para manter as bases atualizadas de modo recorrente:

1- Coleta de Dados Mensais e Anuais

2- Armazenamento Estruturado

3- Atualização Recorrente

4- Verificação de Consistência

5- Consulta e Filtragem:
Ao solicitar dados com o parâmetro A, retornar os dados anuais disponíveis, ou um retorno vazio se ainda não estiverem.
Para consultas com o parâmetro M, retornar os dados mensais disponíveis até a data da consulta.

6- Monitoramento e Alertas

#### Fluxo de Execução
No início de cada mês:

Executar um script para coletar os dados do mês anterior.
Atualizar a tabela de dados mensais com os novos dados.
Verificar a consistência dos dados mensais.

No início de cada trimestre:
Executar um script para verificar se os relatórios anuais de 2024 foram publicados.
Atualize a tabela de dados anuais com os novos relatórios.
Verifique a consistência entre os dados mensais de 2024 e os relatórios anuais de 2024.

Semanalmente:
Execute verificações para garantir que o sistema de coleta e atualização está funcionando corretamente.
Monitore o sistema para detectar quaisquer falhas ou inconsistências.

## 2.Suponha que você precise automatizar o processo de baixar, empilhar e manter atualizado os arquivos de empresas (0~9). Considere que o site é instável, portanto, o arquivo baixado pode estar corrompido.  Liste as etapas necessárias para que o dataset de empresas esteja sempre atualizado.   

1- Criar os diretórios de trabalho

2- Configurar o ambiente em uma Máquina Virtual

3- Baixar os Arquivos

4- Verificar e Validar os Arquivos


5- Descompactar e Processar os Arquivos


6- Juntar os Dados

7- Notificação e Monitoramento

## 3. Escreva uma consulta SQL que retorne, para cada funcionário, o nome do departamento em que ele trabalhou pela primeira vez (baseado na data de início de trabalho), o nome do departamento onde ele está atualmente trabalhando, e a quantidade de departamentos diferentes em que ele já trabalhou.

#### Obter o Primeiro Departamento de Trabalho de Cada Funcionário

In [ ]:
SELECT 
    e.employee_id,
    e.employee_name,
    d.department_name AS first_department
FROM 
    employees e
JOIN 
    employee_department_history edh 
    ON e.employee_id = edh.employee_id
JOIN 
    departments d 
    ON edh.department_id = d.department_id
WHERE 
    edh.start_date = (
        SELECT MIN(start_date) 
        FROM employee_department_history 
        WHERE employee_id = e.employee_id
    );


#### Obter o Departamento de Trabalho Atual de Cada Funcionário

In [ ]:
SELECT 
    e.employee_id,
    e.employee_name,
    d.department_name AS current_department
FROM 
    employees e
JOIN 
    employee_department_history edh 
    ON e.employee_id = edh.employee_id
JOIN 
    departments d 
    ON edh.department_id = d.department_id
WHERE 
    edh.end_date IS NULL;


#### Contar os Departamentos em que Cada Funcionário Já Trabalhou

In [ ]:
SELECT 
    e.employee_id,
    e.employee_name,
    COUNT(DISTINCT edh.department_id) AS num_departments
FROM 
    employees e
JOIN 
    employee_department_history edh 
    ON e.employee_id = edh.employee_id
GROUP BY 
    e.employee_id, e.employee_name;

## 4. O arquivo exemplo1.parquet possui 7GB. O tempo de leitura e retorno de 1 linha específica são 5s - utilizando processamento paralelo (pyspark) em cluster. Contudo, você precisa criar uma API com recursos bem mais modestos, isto é, menor poder de processamento - mantendo o tempo de consulta rápido. Liste as alterações e procedimentos necessários para realizar isso.
### Obs: se necesserário, considere que o arquivo exemplo1.parquet atualiza 1 vez ao mês  

1- Particionar o arquivo parquet

2- Compactar os dados, reduzindo o tamanho físico do arquivo

3- Armazenar resultados de consultas frequentes em cache

## 5. Você foi instruído a criar um modelo de regressão baseado em um modelo econométrico. Ao terminar a implementação você verificou que o resultado está abaixo do esperado. Liste o que poderia ser feito para melhorar o resultado.  

1- Considerar o contexto econômico

2- Verificar a amostragem de dados

3- Reavaliar e ajustar o modelo

4- Verificar variáveis

5- Tratar outliers

## 6. ELT e ETL são termos comuns na rotina de trabalho de especialistas em Soluções de Tecnologia da Informação, justamente por representarem estratégias de pipelines de integração de dados em um determinado projeto. Sobre esses procedimentos, é correto afirmar que:  

Opção Correta: d-  Estrutura de dados em nuvem são ideais para a adoção de estratégias ELT, devido a maior rapidez no carregamento dos dados e a adequada capacidade de processamento posterior.   